https://qiita.com/akita0724/items/694cc4e142d8d1d7fdaa

| 試験区分                         | コード |
|-----------------------------------|------|
| 応用情報技術者試験                 | ap    |
| ITストラテジスト試験               | st    |
| システムアーキテクト試験             | sa    |
| プロジェクトマネージャ試験           | pw    |
| ネットワークスペシャリスト試験       | nw    |
| データベーススペシャリスト試験         | db    |
| エンベデッドシステムスペシャリスト試験 | es    |
| ITサービスマネージャ試験           | sm    |
| システム監査技術者試験             | au    |
| 情報処理安全確保支援士試験           | sc    |

In [8]:
import requests
from os import makedirs, path
from time import sleep
from urllib.request import urlretrieve

# 試験区分とコードの対応表
kamoku_codes = {
    "ap": "応用情報技術者試験",
    "st": "ITストラテジスト試験",
    "sa": "システムアーキテクト試験",
    "pw": "プロジェクトマネージャ試験",
    "nw": "ネットワークスペシャリスト試験",
    "db": "データベーススペシャリスト試験",
    "es": "エンベデッドシステムスペシャリスト試験",
    "sm": "ITサービスマネージャ試験",
    "au": "システム監査技術者試験",
    "sc": "情報処理安全確保支援士試験",
}

# 過去問のURLを生成するための情報
base_url = "https://www.ipa.go.jp/shiken/mondai-kaiotu/"
ids = [
    {"id": "m42obm000000afqx-att", "filename_ids": "2024r06a_"},
    {"id": "m42obm000000afqx-att", "filename_ids": "2024r06h_"},
    {"id": "ps6vr70000010d6y-att", "filename_ids": "2023r05a_"},
    {"id": "ps6vr70000010d6y-att", "filename_ids": "2023r05h_"},
    {"id": "gmcbt80000008smf-att", "filename_ids": "2022r04a_"},
    {"id": "gmcbt80000009sgk-att", "filename_ids": "2022r04h_"},
    {"id": "gmcbt8000000apad-att", "filename_ids": "2021r03a_"},
    {"id": "gmcbt8000000d5ru-att", "filename_ids": "2021r03h_"},
    {"id": "gmcbt8000000d05l-att", "filename_ids": "2020r02o_"},
    {"id": "gmcbt8000000dict-att", "filename_ids": "2019r01a_"},
    {"id": "gmcbt8000000ddiw-att", "filename_ids": "2019h31h_"},
    {"id": "gmcbt8000000f01f-att", "filename_ids": "2018h30a_"},
    {"id": "gmcbt8000000fabr-att", "filename_ids": "2018h30h_"},
    {"id": "gmcbt8000000fqpm-att", "filename_ids": "2017h29a_"},
    {"id": "gmcbt8000000fzx1-att", "filename_ids": "2017h29h_"},
    {"id": "gmcbt8000000g6fw-att", "filename_ids": "2016h28a_"},
    {"id": "gmcbt8000000gn5o-att", "filename_ids": "2016h28h_"},
    {"id": "gmcbt8000000gxj0-att", "filename_ids": "2015h27a_"},
    {"id": "ug65p90000000f52-att", "filename_ids": "2015h27h_"},
    {"id": "ug65p90000000ye5-att", "filename_ids": "2014h26a_"},
    {"id": "ug65p90000001dzu-att", "filename_ids": "2014h26h_"},
    {"id": "ug65p900000027za-att", "filename_ids": "2013h25a_"},
    {"id": "ug65p90000002e6g-att", "filename_ids": "2013h25h_"},
    {"id": "ug65p90000002h5m-att", "filename_ids": "2012h24a_"},
    {"id": "ug65p900000038er-att", "filename_ids": "2012h24h_"},
    {"id": "ug65p90000003ojp-att", "filename_ids": "2011h23a_"},
    {"id": "ug65p90000003ya2-att", "filename_ids": "2011h23tokubetsu_"},
    {"id": "ug65p90000004d6f-att", "filename_ids": "2010h22a_"},
    {"id": "ug65p90000004n2z-att", "filename_ids": "2010h22h_"},
    {"id": "gmcbt8000000f3yi-att", "filename_ids": "2009h21a_"},
    {"id": "ug65p90000009bhl-att", "filename_ids": "2009h21h_"},
]

# ファイルの種類とファイル名のパターン
file_types = {
    "am": ["am_qs", "am_ans"],
    "pm": ["pm_qs", "pm_ans", "pm_cmnt"],
    "am1": ["am1_qs", "am1_ans"],
    "am2": ["am2_qs", "am2_ans"],
    "pm1": ["pm1_qs", "pm1_ans", "pm1_cmnt"],
    "pm2": ["pm2_qs", "pm2_ans", "pm2_cmnt"],
}

# ファイル名のパターン
filename_codes = {
    "qs": "問題",
    "ans": "解答",
    "cmnt": "採点講評",
}


def dl_mondai(kamoku_code):
    kamoku_name = kamoku_codes[kamoku_code]

    for data in ids:
        year = data["filename_ids"][:4]
        season = data["filename_ids"][7]
        season_name = {"h": "春季", "a": "秋季", "o": "特別"}.get(season, "特別")

        create_dir = f"./DL/{kamoku_name}/{year}/{season_name}"
        if not path.exists(create_dir):
            makedirs(create_dir)

        for time_part in ["am", "pm"]:
            # 午前I/II、午後I/IIのファイルがあるか確認
            for i in range(1, 3):
                new_time_part = f"{time_part}{i}"

                # 午前問題の場合、"ap" または "koudo" を使用
                # am1 の場合のみ "koudo" を使用(午前2は各科目名で存在)
                am_code = "koudo" if kamoku_code != "ap" and time_part == "am" and i == 1 else kamoku_code

                # URLとファイル名を生成 (問題)
                new_url_qs = f"{base_url}{data['id']}/{data['filename_ids']}{am_code}_{new_time_part}_qs.pdf"
                new_filename_qs = f"{year}{season_name}-{new_time_part.upper()}_{filename_codes['qs']}.pdf"

                # URLとファイル名を生成 (解答)
                new_url_ans = f"{base_url}{data['id']}/{data['filename_ids']}{am_code}_{new_time_part}_ans.pdf"
                new_filename_ans = f"{year}{season_name}-{new_time_part.upper()}_{filename_codes['ans']}.pdf"

                # 問題をダウンロード
                if dl_file(new_url_qs, f"{create_dir}/{new_filename_qs}"):
                    # 解答をダウンロード
                    dl_file(new_url_ans, f"{create_dir}/{new_filename_ans}")

                    # 採点講評のURLとファイル名を生成 (午後I/IIのみ)
                    if time_part == "pm":
                        new_url_cmnt = f"{base_url}{data['id']}/{data['filename_ids']}{am_code}_{new_time_part}_cmnt.pdf"
                        new_filename_cmnt = f"{year}{season_name}-{new_time_part.upper()}_{filename_codes['cmnt']}.pdf"
                        dl_file(new_url_cmnt, f"{create_dir}/{new_filename_cmnt}")

            # am/pm のファイルがないか確認
            for file_type_suffix in file_types[time_part]:
                # 午前問題の場合、"ap" または "koudo" を使用
                am_code = "koudo" if kamoku_code != "ap" and time_part == "am" else kamoku_code

                # 過去問のURLを生成
                url = f"{base_url}{data['id']}/{data['filename_ids']}{am_code}_{file_type_suffix}.pdf"

                # ファイル名のパターンを決定
                if file_type_suffix.endswith("_cmnt"):
                    filename_pattern_key = "cmnt"  # 採点講評の場合
                else:
                    filename_pattern_key = file_type_suffix.split("_")[1]

                # 保存するファイル名を生成
                filename = f"{year}{season_name}-{time_part.upper()}_{filename_codes[filename_pattern_key]}.pdf"

                # ファイルをダウンロード
                dl_file(url, f"{create_dir}/{filename}")

    print(f"{kamoku_name}のダウンロードが完了しました。")


# ファイルをダウンロードする関数
def dl_file(url, filename):
    if path.exists(filename):
        print(f"{filename} は既に存在するため、ダウンロードをスキップします。")
        return True

    response = requests.head(url)
    if response.status_code == 200:
        urlretrieve(url, filename)
        print(f"{filename} を保存しました。")
        sleep(3)
        return True
    else:
        print(f"{filename} は見つかりませんでした。")
        return False


# プログラムのエントリポイント
if __name__ == "__main__":
    while True:
        inp = input(
            "ダウンロードしたい試験区分のコードを小文字で入力してください（例：ap）：\n修了する場合はqを入力してください。"
        )
        if inp in kamoku_codes:
            dl_mondai(inp)
            break
        elif inp == "q":
            print("終了します。")
            exit()
        else:
            print("試験区分のコードを入力してください。")

./DL/情報処理安全確保支援士試験/2024/秋季/2024秋季-AM1_問題.pdf を保存しました。
./DL/情報処理安全確保支援士試験/2024/秋季/2024秋季-AM1_解答.pdf を保存しました。
./DL/情報処理安全確保支援士試験/2024/秋季/2024秋季-AM2_問題.pdf を保存しました。
./DL/情報処理安全確保支援士試験/2024/秋季/2024秋季-AM2_解答.pdf を保存しました。
./DL/情報処理安全確保支援士試験/2024/秋季/2024秋季-AM_問題.pdf は見つかりませんでした。
./DL/情報処理安全確保支援士試験/2024/秋季/2024秋季-AM_解答.pdf は見つかりませんでした。
./DL/情報処理安全確保支援士試験/2024/秋季/2024秋季-PM1_問題.pdf は見つかりませんでした。
./DL/情報処理安全確保支援士試験/2024/秋季/2024秋季-PM2_問題.pdf は見つかりませんでした。
./DL/情報処理安全確保支援士試験/2024/秋季/2024秋季-PM_問題.pdf を保存しました。
./DL/情報処理安全確保支援士試験/2024/秋季/2024秋季-PM_解答.pdf を保存しました。
./DL/情報処理安全確保支援士試験/2024/秋季/2024秋季-PM_採点講評.pdf は見つかりませんでした。
./DL/情報処理安全確保支援士試験/2024/春季/2024春季-AM1_問題.pdf は既に存在するため、ダウンロードをスキップします。
./DL/情報処理安全確保支援士試験/2024/春季/2024春季-AM1_解答.pdf は既に存在するため、ダウンロードをスキップします。
./DL/情報処理安全確保支援士試験/2024/春季/2024春季-AM2_問題.pdf は既に存在するため、ダウンロードをスキップします。
./DL/情報処理安全確保支援士試験/2024/春季/2024春季-AM2_解答.pdf は既に存在するため、ダウンロードをスキップします。
./DL/情報処理安全確保支援士試験/2024/春季/2024春季-AM_問題.pdf は見つかりませんでした。
./DL/情報処理安全確保支援士試験/2024/春季/2024春季-AM_解答.pdf は見つか

読み取り
https://qiita.com/kccs_hiroshi-tatsuwaki/items/f3277a157acf965c33e8

In [ ]:
!pip install yomitoku

In [7]:
import os
from pdf2image import convert_from_path

# PDFを画像に変換する関数
def convert_pdf_to_image(pdf_path, output_dir_path):
    os.makedirs(output_dir_path, exist_ok=True)

    images = convert_from_path(pdf_path)
    for i, image in enumerate(images):
        output_path = f"{output_dir_path}/page_{i + 1}.png"
        image.save(output_path, "PNG")
        print(f"Saved: {output_path}")

# ダウンロードしたPDFを画像に変換する処理
def process_pdfs():
    dl_dir = "./DL"
    png_dir = "./PNG"
    os.makedirs(png_dir, exist_ok=True)

    for root, dirs, files in os.walk(dl_dir):
        for file in files:
            if file.lower().endswith(".pdf"):
                pdf_path = os.path.join(root, file)
                # 出力先のディレクトリを作成
                relative_path = os.path.relpath(root, dl_dir)
                output_base_dir = os.path.join(png_dir, relative_path)
                
                # ファイル名から拡張子を除いた部分を取得
                file_name_without_ext = os.path.splitext(file)[0]
                
                # ファイル名と同じ名前のフォルダを最下層に作成
                output_dir = os.path.join(output_base_dir, file_name_without_ext)

                # フォルダが存在するか確認
                if os.path.exists(output_dir):
                    print(f"{output_dir} は既に存在するため、変換をスキップします。")
                    continue
                
                os.makedirs(output_dir, exist_ok=True)
                convert_pdf_to_image(pdf_path, output_dir)

if __name__ == "__main__":
    process_pdfs()

./PNG\ITストラテジスト試験\2009\春季\2009春季-AM1_問題 は既に存在するため、変換をスキップします。
./PNG\ITストラテジスト試験\2009\春季\2009春季-AM1_解答 は既に存在するため、変換をスキップします。
./PNG\ITストラテジスト試験\2009\春季\2009春季-AM2_問題 は既に存在するため、変換をスキップします。
./PNG\ITストラテジスト試験\2009\春季\2009春季-AM2_解答 は既に存在するため、変換をスキップします。
./PNG\ITストラテジスト試験\2009\春季\2009春季-PM1_問題 は既に存在するため、変換をスキップします。
./PNG\ITストラテジスト試験\2009\春季\2009春季-PM1_採点講評 は既に存在するため、変換をスキップします。
./PNG\ITストラテジスト試験\2009\春季\2009春季-PM1_解答 は既に存在するため、変換をスキップします。
./PNG\ITストラテジスト試験\2009\春季\2009春季-PM2_問題 は既に存在するため、変換をスキップします。
./PNG\ITストラテジスト試験\2009\春季\2009春季-PM2_採点講評 は既に存在するため、変換をスキップします。
./PNG\ITストラテジスト試験\2009\春季\2009春季-PM2_解答 は既に存在するため、変換をスキップします。
./PNG\ITストラテジスト試験\2009\秋季\2009秋季-AM1_問題 は既に存在するため、変換をスキップします。
./PNG\ITストラテジスト試験\2009\秋季\2009秋季-AM1_解答 は既に存在するため、変換をスキップします。
./PNG\ITストラテジスト試験\2009\秋季\2009秋季-AM2_問題 は既に存在するため、変換をスキップします。
./PNG\ITストラテジスト試験\2009\秋季\2009秋季-AM2_解答 は既に存在するため、変換をスキップします。
./PNG\ITストラテジスト試験\2009\秋季\2009秋季-PM1_問題 は既に存在するため、変換をスキップします。
./PNG\ITストラテジスト試験\2009\秋季\2009秋季-PM1_採点講評 は既に存在するため、変換をスキップします。
./